## 0. Environment
- Venv
- Data decompression

### 0.1 Install python libraries 
> Install pandas and openpyxl in a conda venv

SHELL
```shell
conda create --name pns_venv --yes
conda activate /workspaces/depressao-ibge-pns/pns_venv
conda install pandas --yes
conda install openpyxl --yes
```

### 0.2 Decompress the pns-database and the dictionary
> The data is include in data/compressed-data.zip ~40MB<br>
> Decompresses it and will have two files summing up to ~500MB

SHELL
```shell
unzip data/compressed-data.zip -d data/ -x "__MACOSX/*"
```

## 1. Setup
- Lib Imports
- File paths
- dictionary
- column_widths

### 1.1 Imports

In [1]:
# Libraries
import pandas as pd
import numpy as np

### 1.2 File paths

In [2]:
pns_data_path = "../data/pns-2019.txt"
dictionary_path = "../data/pns-dictionary-clean.xlsx"

### 1.3 Dictionay

In [37]:
dictionary = pd.read_excel(dictionary_path, skiprows=2)
dictionary.head(30)

,ModuloParteAnotacao,PosiçãoInicial,Tamanho,Codigo,Numero,Descricao,Tipo,Categoria
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Parte 1 - Identificação e Controle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,2.0,V0001,NaN,Unidade da Federação,11,Rondônia
3,NaN,NaN,NaN,NaN,NaN,NaN,12,Acre
4,NaN,NaN,NaN,NaN,NaN,NaN,13,Amazonas
5,NaN,NaN,NaN,NaN,NaN,NaN,14,Roraima
6,NaN,NaN,NaN,NaN,NaN,NaN,15,Pará
7,NaN,NaN,NaN,NaN,NaN,NaN,16,Amapá
8,NaN,NaN,NaN,NaN,NaN,NaN,17,Tocantins
9,NaN,NaN,NaN,NaN,NaN,NaN,21,Maranhão


#### 1.3.1 Improve the dictionary and save it as csv

In [38]:
# Remove escape sequences and fill NaN values
dictionary['Descricao'] = dictionary['Descricao'].str.replace(r"\\n", "", regex=True).str.strip()
dictionary['Descricao'] = dictionary['Descricao'].fillna(method='ffill')

# Nullify all the attributes from the dictionary where ModuloParteAnotacao is not null except for ModuloParteAnotacao
dictionary.loc[dictionary['ModuloParteAnotacao'].notnull(), dictionary.columns != 'ModuloParteAnotacao'] = np.nan

# Save the improved dictionary as a CSV file
dictionary.to_csv("../data/dictionary.csv", index=False)

# Display the first few rows to confirm changes
dictionary.head(30)

/tmp/ipykernel_1625/2809857878.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dictionary['Descricao'] = dictionary['Descricao'].fillna(method='ffill')


,ModuloParteAnotacao,PosiçãoInicial,Tamanho,Codigo,Numero,Descricao,Tipo,Categoria
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Parte 1 - Identificação e Controle,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1.0,2.0,V0001,NaN,Unidade da Federação,11,Rondônia
3,NaN,NaN,NaN,NaN,NaN,Unidade da Federação,12,Acre
4,NaN,NaN,NaN,NaN,NaN,Unidade da Federação,13,Amazonas
5,NaN,NaN,NaN,NaN,NaN,Unidade da Federação,14,Roraima
6,NaN,NaN,NaN,NaN,NaN,Unidade da Federação,15,Pará
7,NaN,NaN,NaN,NaN,NaN,Unidade da Federação,16,Amapá
8,NaN,NaN,NaN,NaN,NaN,Unidade da Federação,17,Tocantins
9,NaN,NaN,NaN,NaN,NaN,Unidade da Federação,21,Maranhão


### 1.4 Widths

In [5]:
column_widths = dictionary.Tamanho.dropna(axis=0).values.astype(int)
print(
    column_widths[:10],
    column_widths[1088-10:1088],
)

[2 7 9 4 2 4 2 1 1 1] [1 2 8 8 1 2 1 1 1 8]


## 2. Data Frame
- FWF
- Column descriptions

### 2.1 DF using FWF 

In [6]:
# FWF
df = pd.read_fwf(
    pns_data_path,
    header=None,
    dtype=str,
    widths=column_widths
)

In [7]:
df.shape 

(293726, 1088)

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087
0,11,1110011,110000016,0001,01,2019,06,1,1,1,...,1,06,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
1,11,1110011,110000016,0001,01,2019,06,1,1,0,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
2,11,1110011,110000016,0001,01,2019,06,1,1,0,...,1,04,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
3,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
4,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504


### 2.2 Name DF columns with descriptions from the dictionary

In [9]:
# Descriptions
descriptions = dictionary.Descricao.dropna(axis=0).str.strip()
display(descriptions.head())
display(descriptions.shape)

2                    Unidade da Federação
29                                Estrato
30                                    UPA
31    Número de ordem do domicílio na PNS
32                     Tipo da entrevista
Name: Descricao, dtype: object

(1088,)

In [10]:
df.columns = descriptions
df.head()

Descricao,Unidade da Federação,Estrato,UPA,Número de ordem do domicílio na PNS,Tipo da entrevista,Ano de referência,Total de moradores,Tipo de situação censitária,Tipo de área,Seleção do morador de 15 anos ou mais para responder o questionário individual,...,Condição de ocupação na semana de referência para pessoas de 14 anos ou mais de idade,Grupamentos de atividade do trabalho principal da semana de referência para pessoas de 14 anos ou mais de idade,"Rendimento domiciliar \n(exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)","Rendimento domiciliar per capita\n(exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)","Faixa de rendimento domiciliar per capita (exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)",Idade em meses das crianças nascidas no período de referência do Módulo L,Faixa de tempo gasto por dia no deslocamento casa-trabalho pelas pessoas ocupadas que se deslocavam para o trabalho,Tipo de cigarro industrializado comprado,Método contraceptivo mais eficaz que faz uso,"Data de geração do arquivo de microdados. Data ordenada na forma: ano (4 algarismos), mês (2) e dia (2) - AAAAMMDD"
0,11,1110011,110000016,0001,01,2019,06,1,1,1,...,1,06,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
1,11,1110011,110000016,0001,01,2019,06,1,1,0,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
2,11,1110011,110000016,0001,01,2019,06,1,1,0,...,1,04,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
3,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504
4,11,1110011,110000016,0001,01,2019,06,1,1,9,...,NaN,NaN,00002098,00000350,2,NaN,NaN,NaN,NaN,20220504


## 3. Attribute selection
- Read attribute-selection.txt file
- Remove escape sequences
- `depression` data frame

### 3.1 Read attribute selection

In [11]:
# read data/attribute-selection.txt into a python str list
with open("../data/attribute-selection.txt", "r") as file:
    selected_attributes = [line.replace("\\n", "").strip() for line in file.readlines()]

# show me the duplicates
duplicates = [item for item, count in zip(selected_attributes, np.bincount([selected_attributes.index(i) for i in selected_attributes])) if count > 1]
display(pd.DataFrame(duplicates))

# Remove duplicates from selected_attributes
selected_attributes = list(dict.fromkeys(selected_attributes))
pd.DataFrame(selected_attributes)

""


,0
0,"Nas duas últimas semanas, com que frequência o..."
1,Algum médico ou profissional de saúde mental (...
2,Algum médico já lhe receitou algum medicamento...
3,Nas duas últimas semanas o(a) senhor(a) usou a...
4,Que idade o(a) Sr(a) tinha quando começou a fu...
5,O(A) Sr(a) vai ao médico/serviço de saúde regu...
6,Qual o principal motivo do(a) Sr(a) não visita...
7,Por causa da depressão Faz psicoterapia
8,Por causa da depressão Toma medicamentos
9,Algum dos medicamentos para depressão foi obti...


### 3.2 remove escape \n instances from the df.columns

In [12]:
# remove escaped sequences from the df.columns
df.columns = df.columns.str.replace(r"\n", "", regex=True).str.strip()

### 3.3 Depression V1

In [13]:
depression = df[selected_attributes]
display(depression.shape)
display(depression.head())

(293726, 56)

Descricao,"Nas duas últimas semanas, com que frequência o(a) Sr(a) se sentiu deprimido(a), “pra baixo” ou sem perspectiva?",Algum médico ou profissional de saúde mental (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de depressão?,Algum médico já lhe receitou algum medicamento para depressão?,Nas duas últimas semanas o(a) senhor(a) usou algum medicamento para depressão?,Que idade o(a) Sr(a) tinha quando começou a fumar produto de tabaco diariamente?,O(A) Sr(a) vai ao médico/serviço de saúde regularmente por causa da depressão ou só quando tem algum problema?,Qual o principal motivo do(a) Sr(a) não visitar o médico/serviço de saúde regularmente por causa da depressão?,Por causa da depressão Faz psicoterapia,Por causa da depressão Toma medicamentos,Algum dos medicamentos para depressão foi obtido em serviço público de saúde?,...,Idade do morador na data de referência,Unidade da Federação,Tipo de área,Tipo de situação censitária,Cor ou raça,Qual é a sua orientação sexual?,Qual foi o curso mais elevado que ___frequentou,"Quem fez isso com você? (Se mais de uma pessoa, defina o pirncipal agressor)",Quem fez isso com você,Quem fez isso com você
0,2,1,1,3,15,2,01,2,1,1,...,055,11,1,1,1,1,05,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,069,11,1,1,4,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,031,11,1,1,2,NaN,10,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,009,11,1,1,2,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,006,11,1,1,4,NaN,NaN,NaN,NaN,NaN


## 4. Save to .csv

In [14]:
# Uncomment the following two lines to save
output_path = "../data/depression.csv"
depression.to_csv(output_path, index=False)